In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\R_zia\anaconda3\envs\PythonData\lib\site-packages

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Drop the non-beneficial ID columns, 'EIN' and 'NAME', 'SPECIAL_CONSIDERATIONS'

application_df= application_df.drop(["EIN", "NAME", "SPECIAL_CONSIDERATIONS"], axis=1)
application_df=application_df.dropna()
application_df=application_df[application_df["INCOME_AMT"] > "0"]
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,7508025,1


In [3]:
#APPLICATION COUNT 
application_counts= application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?

replace_application = list(application_counts[application_counts < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       8412
T6        406
T5        314
T4        265
Other     238
T9        146
T7        130
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Generate our categorical variable lists
application_cat=application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#  YOUR CODE GOES HERE
encode_df=pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals

application_df=application_df.merge(encode_df, left_index=True, right_index=True)
application_df=application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T9,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
1,1,108590,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1,31452,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1,7508025,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays

X=application_df.drop(["IS_SUCCESSFUL"],1).values
y=application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 300
hidden_nodes_layer2 = 100

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               24300     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 54,501
Trainable params: 54,501
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# Train the model

fit_model=nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
 576/2079 [=======>......................] - ETA: 3s - loss: 218256.3848 - acc: 0.5556 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
1792/2079 [========================>.....] - ETA: 0s - loss: 212301.9965 - acc: 0.5474
Epoch 00001: saving model to checkpoints/weights.01.hdf5
2079/2079 [==============================] - 2s 1ms/sample - loss: 188223.9681 - acc: 0.5440
Epoch 2/100
 960/2079 [============>.................] - ETA: 0s - loss: 5580.0802 - acc: 0.5531
Epoch 00002: saving model to checkpoints/weights.02.hdf5
1984/2079 [===========================>..] - ETA: 0s - loss: 5135.1946 - acc: 0.5015
Epoch 00002: saving model to checkpoints/weights.02.hdf5
2079/2079 [==============================] - 0s 150us/sample - loss: 5395.8822 - acc: 0.5065
Epoch 3/100
 800/2079 [==========>...................] - ETA: 0s - loss: 9997.3606 - acc: 0.5263 
Epoch 00003: saving model to checkpoints/weights.03.hdf5
1536/2079 [=====================>........] - ETA: 0s - loss: 8

2079/2079 [==============================] - 0s 141us/sample - loss: 17653.6387 - acc: 0.5560
Epoch 22/100
  32/2079 [..............................] - ETA: 0s - loss: 933704.3750 - acc: 0.6875
Epoch 00022: saving model to checkpoints/weights.22.hdf5
1120/2079 [===============>..............] - ETA: 0s - loss: 36527.0673 - acc: 0.4732
Epoch 00022: saving model to checkpoints/weights.22.hdf5
2079/2079 [==============================] - 0s 163us/sample - loss: 20599.6440 - acc: 0.4863
Epoch 23/100
  32/2079 [..............................] - ETA: 0s - loss: 93.4285 - acc: 0.4688
Epoch 00023: saving model to checkpoints/weights.23.hdf5
1248/2079 [=================>............] - ETA: 0s - loss: 2151.1734 - acc: 0.5737
Epoch 00023: saving model to checkpoints/weights.23.hdf5
2079/2079 [==============================] - 0s 142us/sample - loss: 12058.6039 - acc: 0.5806
Epoch 24/100
  32/2079 [..............................] - ETA: 0s - loss: 29156.9043 - acc: 0.5312
Epoch 00024: saving mode

Epoch 00042: saving model to checkpoints/weights.42.hdf5
1536/2079 [=====================>........] - ETA: 0s - loss: 836.5554 - acc: 0.5586
Epoch 00042: saving model to checkpoints/weights.42.hdf5
2079/2079 [==============================] - 0s 169us/sample - loss: 1556.3878 - acc: 0.5599
Epoch 43/100
 448/2079 [=====>........................] - ETA: 0s - loss: 12727.6171 - acc: 0.3772
Epoch 00043: saving model to checkpoints/weights.43.hdf5
1472/2079 [====================>.........] - ETA: 0s - loss: 6982.7842 - acc: 0.4463
Epoch 00043: saving model to checkpoints/weights.43.hdf5
2079/2079 [==============================] - 0s 152us/sample - loss: 7207.6699 - acc: 0.4805
Epoch 44/100
 512/2079 [======>.......................] - ETA: 0s - loss: 8235.6156 - acc: 0.5098
Epoch 00044: saving model to checkpoints/weights.44.hdf5
1440/2079 [===================>..........] - ETA: 0s - loss: 10115.1748 - acc: 0.5417
Epoch 00044: saving model to checkpoints/weights.44.hdf5
2079/2079 [=========

 864/2079 [===========>..................] - ETA: 0s - loss: 827.8725 - acc: 0.4549
Epoch 00063: saving model to checkpoints/weights.63.hdf5
2079/2079 [==============================] - 0s 145us/sample - loss: 806.6169 - acc: 0.5185
Epoch 64/100

Epoch 00064: saving model to checkpoints/weights.64.hdf5
 992/2079 [=============>................] - ETA: 0s - loss: 2189.1518 - acc: 0.5333
Epoch 00064: saving model to checkpoints/weights.64.hdf5
1760/2079 [========================>.....] - ETA: 0s - loss: 8411.8597 - acc: 0.5670
Epoch 00064: saving model to checkpoints/weights.64.hdf5
2079/2079 [==============================] - 1s 249us/sample - loss: 7138.7142 - acc: 0.5604
Epoch 65/100
 544/2079 [======>.......................] - ETA: 0s - loss: 274.5792 - acc: 0.3695
Epoch 00065: saving model to checkpoints/weights.65.hdf5
1920/2079 [==========================>...] - ETA: 0s - loss: 3040.3776 - acc: 0.4760
Epoch 00065: saving model to checkpoints/weights.65.hdf5
2079/2079 [============

1408/2079 [===================>..........] - ETA: 0s - loss: 0.6678 - acc: 0.6129
Epoch 00084: saving model to checkpoints/weights.84.hdf5
2079/2079 [==============================] - 0s 142us/sample - loss: 0.6672 - acc: 0.6142
Epoch 85/100
  32/2079 [..............................] - ETA: 0s - loss: 0.5963 - acc: 0.7812
Epoch 00085: saving model to checkpoints/weights.85.hdf5
1120/2079 [===============>..............] - ETA: 0s - loss: 0.6764 - acc: 0.5920
Epoch 00085: saving model to checkpoints/weights.85.hdf5
2079/2079 [==============================] - 0s 153us/sample - loss: 0.6671 - acc: 0.6142
Epoch 86/100
  32/2079 [..............................] - ETA: 0s - loss: 0.6225 - acc: 0.7188
Epoch 00086: saving model to checkpoints/weights.86.hdf5
1248/2079 [=================>............] - ETA: 0s - loss: 0.6584 - acc: 0.6338
Epoch 00086: saving model to checkpoints/weights.86.hdf5
2079/2079 [==============================] - 0s 153us/sample - loss: 0.6670 - acc: 0.6142
Epoch 87/

In [12]:
#Export our model to HDF5 file
nn.save("AlphabetSoupCharity_OPTIMIZATION1.h5")

In [13]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity_OPTIMIZATION1.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

694/694 - 0s - loss: 0.6887 - acc: 0.5821
Loss: 0.6886538246523063, Accuracy: 0.5821325778961182
